In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

## Загрузка данных

In [2]:
data = pd.read_csv('./data/amine_treatment-final.csv', on_bad_lines='skip')
data

,"feed_gas temperature, C","feed_gas pressure, kPa","feed_gas mass flow, kg/h",feed_gas molecular weight,"feed_gas Mass density, kg/m3","feed_gas actual liquid flow, m3/h","feed_gas actual vapour flow, m3/h",feed_gas CO2 mol frac,feed_gas Methane mol frac,feed_gas Ethane mol frac,...,degased_amine Propane mol frac,degased_amine i-Butane mol frac,degased_amine n-Butane mol frac,degased_amine i-Pentane mol frac,degased_amine n-Pentane mol frac,degased_amine H2S mol frac,degased_amine H2O mol frac,degased_amine MDEAmine mol frac,degased_amine H2S ppm,degased_amine CO2 ppm
0,52.0,6701.325,259647.0,24.668523,80.705482,0.000000,3217.216383,0.031875,0.633122,0.192729,...,0.000004,1.979803e-06,6.785930e-06,3.039502e-08,1.629357e-07,0.014838,0.910916,0.062270,22521.880092,23245.607009
1,40.0,6701.325,63853.0,23.411939,79.292250,0.000000,805.286774,0.039520,0.681947,0.179992,...,0.000006,1.099164e-07,7.886635e-07,2.425456e-06,2.679624e-08,0.010470,0.896612,0.077190,15891.219056,30468.484150
2,46.0,6701.325,15928.0,27.803853,108.358884,0.000000,146.993024,0.027859,0.577870,0.163095,...,0.000018,5.179434e-05,2.863001e-05,4.721545e-06,1.359401e-06,0.001284,0.860268,0.132676,1949.372531,10648.306282
3,36.0,6701.325,150276.0,22.053344,74.092459,0.000000,2028.222609,0.005420,0.856717,0.000249,...,0.000039,8.734358e-06,3.706189e-05,5.426945e-05,3.949909e-06,0.001233,0.809722,0.184359,1871.528366,8438.736973
4,12.0,6701.325,52691.0,24.334510,114.182302,0.002870,461.460940,0.027297,0.694634,0.091063,...,0.000100,7.719691e-05,2.661312e-05,1.351350e-05,6.619552e-06,0.003300,0.788340,0.200715,5008.207346,14011.150105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512795,18.0,6701.325,278813.0,27.267819,147.437736,0.239690,1890.816202,0.036621,0.666807,0.042159,...,0.000053,1.226987e-04,3.336251e-05,4.145371e-06,8.510407e-06,0.002101,0.796328,0.187149,3188.443718,27411.897443
512796,33.0,6701.325,72612.0,26.629816,112.530303,0.000000,645.266193,0.025383,0.605142,0.150361,...,0.000049,8.271254e-05,1.199104e-05,1.810930e-06,4.749955e-06,0.003784,0.826966,0.162727,5742.847844,11941.343733
512797,17.0,6701.325,272718.0,26.290335,135.390018,0.133398,2014.180541,0.039356,0.606850,0.167318,...,0.000037,2.874297e-05,3.343060e-06,1.045919e-05,2.187381e-06,0.045335,0.725301,0.179546,68812.636471,97071.672175
512798,59.0,6701.325,75970.0,24.231673,73.727565,0.000000,1030.415146,0.037951,0.732049,0.086286,...,0.000013,1.782460e-05,8.286743e-06,5.417825e-06,2.393693e-07,0.002551,0.877567,0.106280,3871.441495,26287.366187


In [3]:
list(data.columns)

['feed_gas temperature, C',
 'feed_gas pressure, kPa',
 'feed_gas mass flow, kg/h',
 'feed_gas molecular weight',
 'feed_gas Mass density, kg/m3',
 'feed_gas actual liquid flow, m3/h',
 'feed_gas actual vapour flow, m3/h',
 'feed_gas CO2 mol frac',
 'feed_gas Methane mol frac',
 'feed_gas Ethane mol frac',
 'feed_gas Propane mol frac',
 'feed_gas i-Butane mol frac',
 'feed_gas n-Butane mol frac',
 'feed_gas i-Pentane mol frac',
 'feed_gas n-Pentane mol frac',
 'feed_gas H2S mol frac',
 'feed_gas H2O mol frac',
 'feed_gas MDEAmine mol frac',
 'feed_gas H2S ppm',
 'feed_gas CO2 ppm',
 'Absorber number of stages',
 'lean_amine temperature, C',
 'lean_amine pressure, kPa',
 'lean_amine mass flow, kg/h',
 'lean_amine molecular weight',
 'lean_amine Mass density, kg/m3',
 'lean_amine actual liquid flow, m3/h',
 'lean_amine actual vapour flow, m3/h',
 'lean_amine CO2 mol frac',
 'lean_amine Methane mol frac',
 'lean_amine Ethane mol frac',
 'lean_amine Propane mol frac',
 'lean_amine i-Butane

In [4]:
data['rich_amine molar flow, kgmol/h'] = data['rich_amine mass flow, kg/h'] / data['rich_amine molecular weight']
data['rich_amine H2S molar flow, kgmol/h'] = data['rich_amine molar flow, kgmol/h'] * data['rich_amine H2S mol frac']
data['rich_amine CO2 molar flow, kgmol/h'] = data['rich_amine molar flow, kgmol/h'] * data['rich_amine CO2 mol frac']
data['rich_amine H2O molar flow, kgmol/h'] = data['rich_amine molar flow, kgmol/h'] * data['rich_amine H2O mol frac']
data['rich_amine MDEAmine molar flow, kgmol/h'] = data['rich_amine molar flow, kgmol/h'] * data['rich_amine MDEAmine mol frac']

data['feed_gas molar flow, kgmol/h'] = data['feed_gas mass flow, kg/h'] / data['feed_gas molecular weight']
data['feed_gas H2S molar flow, kgmol/h'] = data['feed_gas molar flow, kgmol/h'] * data['feed_gas H2S mol frac']
data['feed_gas CO2 molar flow, kgmol/h'] = data['feed_gas molar flow, kgmol/h'] * data['feed_gas CO2 mol frac']
data['feed_gas H2O molar flow, kgmol/h'] = data['feed_gas molar flow, kgmol/h'] * data['feed_gas H2O mol frac']
data['feed_gas MDEAmine molar flow, kgmol/h'] = data['feed_gas molar flow, kgmol/h'] * data['feed_gas MDEAmine mol frac']


data['lean_amine molar flow, kgmol/h'] = data['lean_amine mass flow, kg/h'] / data['lean_amine molecular weight']
data['lean_amine H2S molar flow, kgmol/h'] = data['lean_amine molar flow, kgmol/h'] * data['lean_amine H2S mol frac']
data['lean_amine CO2 molar flow, kgmol/h'] = data['lean_amine molar flow, kgmol/h'] * data['lean_amine CO2 mol frac']
data['lean_amine H2O molar flow, kgmol/h'] = data['lean_amine molar flow, kgmol/h'] * data['lean_amine H2O mol frac']
data['lean_amine MDEAmine molar flow, kgmol/h'] = data['lean_amine molar flow, kgmol/h'] * data['lean_amine MDEAmine mol frac']

## Модель для определения температуры продуктов

In [3]:
mini_data = data.sample(frac=0.25)
len(mini_data)

128200

In [6]:
sulphur = data[['feed_gas temperature, C', 'feed_gas mass flow, kg/h', 'feed_gas CO2 mol frac',
                'feed_gas Methane mol frac', 'feed_gas Ethane mol frac', 'feed_gas Propane mol frac', 
                'feed_gas i-Butane mol frac', 'feed_gas n-Butane mol frac', 'feed_gas i-Pentane mol frac',
                'feed_gas n-Pentane mol frac', 'feed_gas H2S mol frac', 'feed_gas H2O mol frac',
                'feed_gas MDEAmine mol frac',
                'lean_amine temperature, C', 'lean_amine mass flow, kg/h', 'lean_amine CO2 mol frac',
                'lean_amine Methane mol frac', 'lean_amine Ethane mol frac', 'lean_amine Propane mol frac',
                'lean_amine i-Butane mol frac', 'lean_amine n-Butane mol frac', 'lean_amine i-Pentane mol frac',
                'lean_amine n-Pentane mol frac', 'lean_amine H2S mol frac', 'lean_amine H2O mol frac',
                'lean_amine MDEAmine mol frac', 'sweet_gas temperature, C', 'rich_amine temperature, C'
               ]]

X = sulphur[[
                'feed_gas temperature, C', 'feed_gas mass flow, kg/h', 'feed_gas CO2 mol frac',
                'feed_gas Methane mol frac', 'feed_gas Ethane mol frac', 'feed_gas Propane mol frac', 
                'feed_gas i-Butane mol frac', 'feed_gas n-Butane mol frac', 'feed_gas i-Pentane mol frac',
                'feed_gas n-Pentane mol frac', 'feed_gas H2S mol frac', 'feed_gas H2O mol frac',
                'feed_gas MDEAmine mol frac',
                'lean_amine temperature, C', 'lean_amine mass flow, kg/h', 'lean_amine CO2 mol frac',
                'lean_amine Methane mol frac', 'lean_amine Ethane mol frac', 'lean_amine Propane mol frac',
                'lean_amine i-Butane mol frac', 'lean_amine n-Butane mol frac', 'lean_amine i-Pentane mol frac',
                'lean_amine n-Pentane mol frac', 'lean_amine H2S mol frac', 'lean_amine H2O mol frac',
                'lean_amine MDEAmine mol frac'
]]

y = sulphur[['sweet_gas temperature, C', 'rich_amine temperature, C']]

# Собираем трансформер столбцов
ct = make_column_transformer(
    (MinMaxScaler(), ['feed_gas temperature, C', 'feed_gas mass flow, kg/h', 'feed_gas CO2 mol frac',
                'feed_gas Methane mol frac', 'feed_gas Ethane mol frac', 'feed_gas Propane mol frac', 
                'feed_gas i-Butane mol frac', 'feed_gas n-Butane mol frac', 'feed_gas i-Pentane mol frac',
                'feed_gas n-Pentane mol frac', 'feed_gas H2S mol frac', 'feed_gas H2O mol frac',
                'feed_gas MDEAmine mol frac',
                'lean_amine temperature, C', 'lean_amine mass flow, kg/h', 'lean_amine CO2 mol frac',
                'lean_amine Methane mol frac', 'lean_amine Ethane mol frac', 'lean_amine Propane mol frac',
                'lean_amine i-Butane mol frac', 'lean_amine n-Butane mol frac', 'lean_amine i-Pentane mol frac',
                'lean_amine n-Pentane mol frac', 'lean_amine H2S mol frac', 'lean_amine H2O mol frac',
                'lean_amine MDEAmine mol frac'])
)

# Создаем обучающую и тестовые выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Загружаем тренировочную выборку
ct.fit(X_train)

# Нормализуем данные
X_train_normal = ct.transform(X_train)
X_test_normal = ct.transform(X_test)

X_train_normal.shape, y_train.shape

((410240, 26), (410240, 2))

In [86]:
# Set random set
tf.random.set_seed(42)

# 1. Create a model
prod_temp_model = tf.keras.Sequential([
    tf.keras.layers.Dense(26, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(2)
])

# 2. Compile the model
prod_temp_model.compile(
    loss='mae',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['mae']
)

# 3. Fit the model
prod_temp_model.fit(X_train_normal, y_train, epochs=20)

Epoch 1/20
12820/12820 [==============================] - 31s 2ms/step - loss: 2.8807 - mae: 2.8807
Epoch 2/20
12820/12820 [==============================] - 31s 2ms/step - loss: 1.7489 - mae: 1.7489
Epoch 3/20
12820/12820 [==============================] - 31s 2ms/step - loss: 1.5962 - mae: 1.5962
Epoch 4/20
12820/12820 [==============================] - 32s 2ms/step - loss: 1.4959 - mae: 1.4959
Epoch 5/20
12820/12820 [==============================] - 31s 2ms/step - loss: 1.4451 - mae: 1.4451
Epoch 6/20
12820/12820 [==============================] - 32s 2ms/step - loss: 1.4027 - mae: 1.4027
Epoch 7/20
12820/12820 [==============================] - 31s 2ms/step - loss: 1.3708 - mae: 1.3708
Epoch 8/20
12820/12820 [==============================] - 32s 2ms/step - loss: 1.3418 - mae: 1.3418
Epoch 9/20
12820/12820 [==============================] - 31s 2ms/step - loss: 1.3195 - mae: 1.3195
Epoch 10/20
12820/12820 [==============================] - 32s 2ms/step - loss: 1.3019 - mae: 1.3019

In [87]:
y_pred = prod_temp_model.predict(X_test_normal)

3205/3205 [==============================] - 4s 1ms/step


In [107]:
y_pred[101]

array([57.381298, 55.735657], dtype=float32)

In [108]:
y_test.iloc[101]

sweet_gas temperature, C     57.480083
rich_amine temperature, C    55.968702
Name: 237204, dtype: float64

In [110]:
100 - ((55.735657 * 100) / 55.968702)

0.4163845000371822

In [111]:
prod_temp_model.save('./models/amine_treatment/amine_treatment_prod_temp_model.h5')

In [12]:
sulphur_data = sulphur.sample(frac=0.01)
sulphur_data

,"feed_gas temperature, C","feed_gas mass flow, kg/h",feed_gas CO2 mol frac,feed_gas Methane mol frac,feed_gas Ethane mol frac,feed_gas Propane mol frac,feed_gas i-Butane mol frac,feed_gas n-Butane mol frac,feed_gas i-Pentane mol frac,feed_gas n-Pentane mol frac,...,lean_amine Propane mol frac,lean_amine i-Butane mol frac,lean_amine n-Butane mol frac,lean_amine i-Pentane mol frac,lean_amine n-Pentane mol frac,lean_amine H2S mol frac,lean_amine H2O mol frac,lean_amine MDEAmine mol frac,"sweet_gas temperature, C","rich_amine temperature, C"
229777,59.0,295333.0,0.037486,0.709195,0.027945,0.049124,0.096944,0.016101,0.022952,0.004392,...,0.0,0.0,0.0,0.0,0.0,0.000211,0.977453,0.019753,63.534447,65.537233
230248,22.0,51765.0,0.036442,0.649914,0.060758,0.072099,0.052338,0.046696,0.041313,0.003336,...,0.0,0.0,0.0,0.0,0.0,0.000344,0.810059,0.184826,25.056876,30.451538
142441,53.0,94584.0,0.026240,0.633435,0.186715,0.090658,0.019003,0.011180,0.025023,0.002668,...,0.0,0.0,0.0,0.0,0.0,0.000291,0.890271,0.108304,26.411256,36.111568
236140,39.0,135235.0,0.028697,0.800068,0.024626,0.082167,0.009194,0.023052,0.017758,0.006773,...,0.0,0.0,0.0,0.0,0.0,0.000181,0.839054,0.157503,35.072823,45.195079
328006,50.0,27226.0,0.024775,0.728327,0.096035,0.039802,0.018610,0.062545,0.017170,0.009563,...,0.0,0.0,0.0,0.0,0.0,0.000238,0.899695,0.097257,40.644903,46.896273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127774,57.0,106247.0,0.007134,0.564712,0.150123,0.092254,0.052006,0.076666,0.035214,0.003127,...,0.0,0.0,0.0,0.0,0.0,0.000139,0.860390,0.136670,33.316939,47.659560
340722,12.0,139188.0,0.014490,0.654710,0.126441,0.074161,0.058075,0.011644,0.014428,0.007704,...,0.0,0.0,0.0,0.0,0.0,0.000116,0.858768,0.137278,23.306435,25.228895
460614,20.0,278135.0,0.020406,0.661170,0.083639,0.022305,0.068197,0.078623,0.049696,0.000717,...,0.0,0.0,0.0,0.0,0.0,0.000386,0.845761,0.152531,21.611649,26.587039
437188,43.0,94367.0,0.028831,0.658443,0.177795,0.036555,0.024317,0.051922,0.007627,0.000021,...,0.0,0.0,0.0,0.0,0.0,0.000317,0.980460,0.017175,32.290935,35.745604


In [13]:
sulphur_data.to_csv('sulphur_data.csv')

## Модель для определения массовых расходов продуктовых потоков

In [12]:
mini_data = data.sample(frac=0.85)
len(mini_data)

435880

In [13]:
prod_mass_flow = data[[
    'feed_gas temperature, C', 'feed_gas mass flow, kg/h', 'feed_gas CO2 mol frac', 'feed_gas Methane mol frac',
    'feed_gas Ethane mol frac', 'feed_gas Propane mol frac', 'feed_gas i-Butane mol frac', 'feed_gas n-Butane mol frac',
    'feed_gas i-Pentane mol frac', 'feed_gas n-Pentane mol frac', 'feed_gas H2S mol frac', 'feed_gas H2O mol frac',
    'feed_gas MDEAmine mol frac', 'lean_amine temperature, C', 'lean_amine mass flow, kg/h', 'lean_amine CO2 mol frac',
    'lean_amine Methane mol frac', 'lean_amine Ethane mol frac', 'lean_amine Propane mol frac',
    'lean_amine i-Butane mol frac', 'lean_amine n-Butane mol frac', 'lean_amine i-Pentane mol frac',
    'lean_amine n-Pentane mol frac', 'lean_amine H2S mol frac', 'lean_amine H2O mol frac', 'lean_amine MDEAmine mol frac',
    'sweet_gas temperature, C', 'rich_amine temperature, C',
    'rich_amine mass flow, kg/h', 'sweet_gas mass flow, kg/h'
]]

X = prod_mass_flow[[
    'feed_gas temperature, C', 'feed_gas mass flow, kg/h', 'feed_gas CO2 mol frac', 'feed_gas Methane mol frac',
    'feed_gas Ethane mol frac', 'feed_gas Propane mol frac', 'feed_gas i-Butane mol frac', 'feed_gas n-Butane mol frac',
    'feed_gas i-Pentane mol frac', 'feed_gas n-Pentane mol frac', 'feed_gas H2S mol frac', 'feed_gas H2O mol frac',
    'feed_gas MDEAmine mol frac', 'lean_amine temperature, C', 'lean_amine mass flow, kg/h', 'lean_amine CO2 mol frac',
    'lean_amine Methane mol frac', 'lean_amine Ethane mol frac', 'lean_amine Propane mol frac',
    'lean_amine i-Butane mol frac', 'lean_amine n-Butane mol frac', 'lean_amine i-Pentane mol frac',
    'lean_amine n-Pentane mol frac', 'lean_amine H2S mol frac', 'lean_amine H2O mol frac', 'lean_amine MDEAmine mol frac',
    'sweet_gas temperature, C', 'rich_amine temperature, C',
]]

y = prod_mass_flow[['rich_amine mass flow, kg/h']]

# Собираем трансформер столбцов
ct = make_column_transformer(
    (MinMaxScaler(), [
    'feed_gas temperature, C', 'feed_gas mass flow, kg/h', 'feed_gas CO2 mol frac', 'feed_gas Methane mol frac',
    'feed_gas Ethane mol frac', 'feed_gas Propane mol frac', 'feed_gas i-Butane mol frac', 'feed_gas n-Butane mol frac',
    'feed_gas i-Pentane mol frac', 'feed_gas n-Pentane mol frac', 'feed_gas H2S mol frac', 'feed_gas H2O mol frac',
    'feed_gas MDEAmine mol frac', 'lean_amine temperature, C', 'lean_amine mass flow, kg/h', 'lean_amine CO2 mol frac',
    'lean_amine Methane mol frac', 'lean_amine Ethane mol frac', 'lean_amine Propane mol frac',
    'lean_amine i-Butane mol frac', 'lean_amine n-Butane mol frac', 'lean_amine i-Pentane mol frac',
    'lean_amine n-Pentane mol frac', 'lean_amine H2S mol frac', 'lean_amine H2O mol frac', 'lean_amine MDEAmine mol frac',
    'sweet_gas temperature, C', 'rich_amine temperature, C',
    ])
)

# Создаем обучающую и тестовые выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Загружаем тренировочную выборку
ct.fit(X_train)

# Нормализуем данные
X_train_normal = ct.transform(X_train)
X_test_normal = ct.transform(X_test)

X_train_normal.shape, y_train.shape

((410240, 28), (410240, 1))

In [114]:
# Set random set
tf.random.set_seed(42)

# 1. Create a model
rich_amine_mass_flow_model = tf.keras.Sequential([
    tf.keras.layers.Dense(28, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(1)
])

# 2. Compile the model
rich_amine_mass_flow_model.compile(
    loss='mae',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['mae']
)

# 3. Fit the model
rich_amine_mass_flow_model.fit(X_train_normal, y_train, epochs=20)

Epoch 1/20
12820/12820 [==============================] - 35s 3ms/step - loss: 14747.3662 - mae: 14747.3662
Epoch 2/20
12820/12820 [==============================] - 34s 3ms/step - loss: 1565.1855 - mae: 1565.1855
Epoch 3/20
12820/12820 [==============================] - 33s 3ms/step - loss: 1444.7600 - mae: 1444.7600
Epoch 4/20
12820/12820 [==============================] - 33s 3ms/step - loss: 1418.8979 - mae: 1418.8979
Epoch 5/20
12820/12820 [==============================] - 35s 3ms/step - loss: 1412.1162 - mae: 1412.1162
Epoch 6/20
12820/12820 [==============================] - 33s 3ms/step - loss: 1407.0701 - mae: 1407.0701
Epoch 7/20
12820/12820 [==============================] - 33s 3ms/step - loss: 1391.9468 - mae: 1391.9468
Epoch 8/20
12820/12820 [==============================] - 32s 2ms/step - loss: 1382.7007 - mae: 1382.7007
Epoch 9/20
12820/12820 [==============================] - 34s 3ms/step - loss: 1372.9044 - mae: 1372.9044
Epoch 10/20
12820/12820 [===================

In [115]:
y_pred = rich_amine_mass_flow_model.predict(X_test_normal)

3205/3205 [==============================] - 4s 1ms/step


In [120]:
x = 1098
y_pred[x]

array([325158.9], dtype=float32)

In [121]:
y_test.iloc[x]

rich_amine mass flow, kg/h    325148.650133
Name: 478285, dtype: float64

In [119]:
(543540.56 * 100) / 544287.473182

99.86277229978612

In [122]:
rich_amine_mass_flow_model.save('./models/amine_treatment/amine_treatment_rich_amine_mass_flow.h5')

In [15]:
rich_amine_mass_flow_data = prod_mass_flow.sample(frac=0.01)
rich_amine_mass_flow_data

,"feed_gas temperature, C","feed_gas mass flow, kg/h",feed_gas CO2 mol frac,feed_gas Methane mol frac,feed_gas Ethane mol frac,feed_gas Propane mol frac,feed_gas i-Butane mol frac,feed_gas n-Butane mol frac,feed_gas i-Pentane mol frac,feed_gas n-Pentane mol frac,...,lean_amine n-Butane mol frac,lean_amine i-Pentane mol frac,lean_amine n-Pentane mol frac,lean_amine H2S mol frac,lean_amine H2O mol frac,lean_amine MDEAmine mol frac,"sweet_gas temperature, C","rich_amine temperature, C","rich_amine mass flow, kg/h","sweet_gas mass flow, kg/h"
378785,34.0,43352.0,0.028730,0.730447,0.055514,0.099421,0.025227,0.001636,0.017257,0.008542,...,0.0,0.0,0.0,0.000222,0.828388,0.167117,34.640048,44.629556,172799.511011,39440.488989
118926,41.0,169862.0,0.027947,0.741430,0.005786,0.021576,0.059063,0.082672,0.021086,0.002751,...,0.0,0.0,0.0,0.000306,0.943323,0.055319,40.024375,40.382760,31566.931899,167568.068101
484677,19.0,200299.0,0.035638,0.743175,0.002663,0.036137,0.034679,0.079880,0.045699,0.009657,...,0.0,0.0,0.0,0.000229,0.791924,0.206932,43.867206,42.554392,357737.285549,186419.714451
172004,37.0,15602.0,0.038805,0.632071,0.137268,0.029537,0.032185,0.080976,0.027329,0.001191,...,0.0,0.0,0.0,0.000115,0.963816,0.034689,49.918045,53.411792,84879.752815,14220.247185
373724,18.0,276450.0,0.035284,0.517867,0.154605,0.041304,0.064756,0.092810,0.045651,0.008637,...,0.0,0.0,0.0,0.000143,0.974992,0.022430,34.287812,24.770478,496446.706692,256522.293308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98277,29.0,160620.0,0.014560,0.737626,0.059342,0.011977,0.099359,0.038867,0.029087,0.002439,...,0.0,0.0,0.0,0.000156,0.790771,0.208055,36.000133,36.903628,634442.178968,155221.821032
179710,22.0,196540.0,0.019006,0.726973,0.005725,0.074952,0.027308,0.098185,0.008860,0.004578,...,0.0,0.0,0.0,0.000247,0.810191,0.188147,17.920779,45.870836,222515.108397,181618.891603
158766,45.0,284251.0,0.009030,0.767586,0.016812,0.053575,0.048981,0.028589,0.045822,0.009651,...,0.0,0.0,0.0,0.000342,0.752415,0.243823,45.985001,57.590519,354210.540020,273610.459980
298377,59.0,118392.0,0.007394,0.757411,0.025888,0.020194,0.060058,0.061593,0.037414,0.001076,...,0.0,0.0,0.0,0.000333,0.755419,0.242670,16.736872,66.343628,154380.854558,112142.145442


In [16]:
rich_amine_mass_flow_data.to_csv('rich_amine_mass_flow_data.csv')

## Модель для определения молярного расхода H2S и CO2 в продуктовых потоках

In [77]:
mini_data = data.sample(frac=0.95)
len(mini_data)

487160

In [96]:
sulphur = mini_data[[
    'feed_gas temperature, C', 'feed_gas mass flow, kg/h', 'feed_gas CO2 mol frac', 'feed_gas Methane mol frac',
    'feed_gas Ethane mol frac', 'feed_gas Propane mol frac', 'feed_gas i-Butane mol frac', 'feed_gas n-Butane mol frac',
    'feed_gas i-Pentane mol frac', 'feed_gas n-Pentane mol frac', 'feed_gas H2S mol frac', 'feed_gas H2O mol frac',
    'feed_gas MDEAmine mol frac', 
    'lean_amine temperature, C', 'lean_amine mass flow, kg/h', 'lean_amine CO2 mol frac',
    'lean_amine Methane mol frac', 'lean_amine Ethane mol frac', 'lean_amine Propane mol frac',
    'lean_amine i-Butane mol frac', 'lean_amine n-Butane mol frac', 'lean_amine i-Pentane mol frac',
    'lean_amine n-Pentane mol frac', 'lean_amine H2S mol frac', 'lean_amine H2O mol frac', 'lean_amine MDEAmine mol frac',
    'feed_gas molar flow, kgmol/h', 'feed_gas H2S molar flow, kgmol/h', 'feed_gas CO2 molar flow, kgmol/h',
    'lean_amine molar flow, kgmol/h', 'lean_amine H2S molar flow, kgmol/h', 'lean_amine CO2 molar flow, kgmol/h',
    'rich_amine temperature, C', 'rich_amine mass flow, kg/h', 'rich_amine molecular weight', 
    'rich_amine molar flow, kgmol/h', 'rich_amine H2S molar flow, kgmol/h', 'rich_amine CO2 molar flow, kgmol/h'
]]

In [97]:
X = sulphur[[
    'feed_gas temperature, C', 'feed_gas mass flow, kg/h', 'feed_gas CO2 mol frac', 'feed_gas Methane mol frac',
    'feed_gas Ethane mol frac', 'feed_gas Propane mol frac', 'feed_gas i-Butane mol frac', 'feed_gas n-Butane mol frac',
    'feed_gas i-Pentane mol frac', 'feed_gas n-Pentane mol frac', 'feed_gas H2S mol frac', 'feed_gas H2O mol frac',
    'feed_gas MDEAmine mol frac', 
    'lean_amine temperature, C', 'lean_amine mass flow, kg/h', 'lean_amine CO2 mol frac',
    'lean_amine Methane mol frac', 'lean_amine Ethane mol frac', 'lean_amine Propane mol frac',
    'lean_amine i-Butane mol frac', 'lean_amine n-Butane mol frac', 'lean_amine i-Pentane mol frac',
    'lean_amine n-Pentane mol frac', 'lean_amine H2S mol frac', 'lean_amine H2O mol frac', 'lean_amine MDEAmine mol frac',
    'feed_gas molar flow, kgmol/h', 'feed_gas H2S molar flow, kgmol/h', 'feed_gas CO2 molar flow, kgmol/h',
    'lean_amine molar flow, kgmol/h', 'lean_amine H2S molar flow, kgmol/h', 'lean_amine CO2 molar flow, kgmol/h',
    'rich_amine temperature, C', 'rich_amine mass flow, kg/h', 'rich_amine molecular weight',
    'rich_amine molar flow, kgmol/h'
]]

y = sulphur[['rich_amine H2S molar flow, kgmol/h', 'rich_amine CO2 molar flow, kgmol/h']]

# Собираем трансформер столбцов
ct = make_column_transformer(
    (MinMaxScaler(), [
    'feed_gas temperature, C', 'feed_gas mass flow, kg/h', 'feed_gas CO2 mol frac', 'feed_gas Methane mol frac',
    'feed_gas Ethane mol frac', 'feed_gas Propane mol frac', 'feed_gas i-Butane mol frac', 'feed_gas n-Butane mol frac',
    'feed_gas i-Pentane mol frac', 'feed_gas n-Pentane mol frac', 'feed_gas H2S mol frac', 'feed_gas H2O mol frac',
    'feed_gas MDEAmine mol frac', 
    'lean_amine temperature, C', 'lean_amine mass flow, kg/h', 'lean_amine CO2 mol frac',
    'lean_amine Methane mol frac', 'lean_amine Ethane mol frac', 'lean_amine Propane mol frac',
    'lean_amine i-Butane mol frac', 'lean_amine n-Butane mol frac', 'lean_amine i-Pentane mol frac',
    'lean_amine n-Pentane mol frac', 'lean_amine H2S mol frac', 'lean_amine H2O mol frac', 'lean_amine MDEAmine mol frac',
    'feed_gas molar flow, kgmol/h', 'feed_gas H2S molar flow, kgmol/h', 'feed_gas CO2 molar flow, kgmol/h',
    'lean_amine molar flow, kgmol/h', 'lean_amine H2S molar flow, kgmol/h', 'lean_amine CO2 molar flow, kgmol/h',
    'rich_amine temperature, C', 'rich_amine mass flow, kg/h', 'rich_amine molecular weight',
    'rich_amine molar flow, kgmol/h'
    ])
)

# Создаем обучающую и тестовые выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Загружаем тренировочную выборку
ct.fit(X_train)

# Нормализуем данные
X_train_normal = ct.transform(X_train)
X_test_normal = ct.transform(X_test)

X_train_normal.shape, y_train.shape

((389728, 36), (389728, 2))

In [99]:
# Set random set
tf.random.set_seed(42)

# 1. Create a model
prod_sour_comp_molar_flow_model = tf.keras.Sequential([
    tf.keras.layers.Dense(36, activation='relu'),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(2)
])

# 2. Compile the model
prod_sour_comp_molar_flow_model.compile(
    loss='mae',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['mae']
)

# 3. Fit the model
prod_sour_comp_molar_flow_model.fit(X_train_normal, y_train, epochs=100)

Epoch 1/100
12179/12179 [==============================] - 29s 2ms/step - loss: 15.6867 - mae: 15.6867
Epoch 2/100
12179/12179 [==============================] - 25s 2ms/step - loss: 11.8431 - mae: 11.8431
Epoch 3/100
12179/12179 [==============================] - 26s 2ms/step - loss: 11.4663 - mae: 11.4663
Epoch 4/100
12179/12179 [==============================] - 27s 2ms/step - loss: 10.8892 - mae: 10.8892
Epoch 5/100
12179/12179 [==============================] - 27s 2ms/step - loss: 10.0814 - mae: 10.0814
Epoch 6/100
12179/12179 [==============================] - 26s 2ms/step - loss: 9.1699 - mae: 9.1699
Epoch 7/100
12179/12179 [==============================] - 26s 2ms/step - loss: 8.2739 - mae: 8.2739
Epoch 8/100
12179/12179 [==============================] - 26s 2ms/step - loss: 7.6678 - mae: 7.6678
Epoch 9/100
12179/12179 [==============================] - 27s 2ms/step - loss: 7.3628 - mae: 7.3628
Epoch 10/100
12179/12179 [==============================] - 28s 2ms/step - loss: 

12179/12179 [==============================] - 26s 2ms/step - loss: 3.5221 - mae: 3.5221
Epoch 82/100
12179/12179 [==============================] - 26s 2ms/step - loss: 3.4622 - mae: 3.4622
Epoch 83/100
12179/12179 [==============================] - 24s 2ms/step - loss: 3.4378 - mae: 3.4378
Epoch 84/100
12179/12179 [==============================] - 27s 2ms/step - loss: 3.3913 - mae: 3.3913
Epoch 85/100
12179/12179 [==============================] - 27s 2ms/step - loss: 3.4307 - mae: 3.4307
Epoch 86/100
12179/12179 [==============================] - 26s 2ms/step - loss: 3.3818 - mae: 3.3818
Epoch 87/100
12179/12179 [==============================] - 27s 2ms/step - loss: 3.3808 - mae: 3.3808
Epoch 88/100
12179/12179 [==============================] - 26s 2ms/step - loss: 3.3696 - mae: 3.3696
Epoch 89/100
12179/12179 [==============================] - 26s 2ms/step - loss: 3.3462 - mae: 3.3462
Epoch 90/100
12179/12179 [==============================] - 26s 2ms/step - loss: 3.3500 - mae: 

In [116]:
y_pred = prod_sour_comp_molar_flow_model.predict(X_test_normal)

3045/3045 [==============================] - 4s 1ms/step


In [149]:
x = 6783
pred = y_pred[x]
pred

array([152.65236,  96.81318], dtype=float32)

In [150]:
test = y_test.iloc[x]
test

rich_amine H2S molar flow, kgmol/h    152.735390
rich_amine CO2 molar flow, kgmol/h     97.016812
Name: 1347, dtype: float64

In [151]:
100 - (pred * 100) / test

rich_amine H2S molar flow, kgmol/h    0.05436
rich_amine CO2 molar flow, kgmol/h    0.20989
Name: 1347, dtype: float64

In [130]:
prod_sour_comp_molar_flow_model.save('./models/amine_treatment/amine_treatment_prod_sour_comp_molar_flow_model.h5')

In [8]:
mini_sulphur = sulphur.sample(frac=0.01)
mini_sulphur

,"feed_gas temperature, C","feed_gas mass flow, kg/h",feed_gas CO2 mol frac,feed_gas Methane mol frac,feed_gas Ethane mol frac,feed_gas Propane mol frac,feed_gas i-Butane mol frac,feed_gas n-Butane mol frac,feed_gas i-Pentane mol frac,feed_gas n-Pentane mol frac,...,"lean_amine H2S molar flow, kgmol/h","lean_amine CO2 molar flow, kgmol/h","rich_amine temperature, C","rich_amine mass flow, kg/h",rich_amine molecular weight,rich_amine H2S mol frac,rich_amine CO2 mol frac,"rich_amine molar flow, kgmol/h","rich_amine H2S molar flow, kgmol/h","rich_amine CO2 molar flow, kgmol/h"
164,51.0,201156.0,0.029106,0.531202,0.190802,0.006178,0.088199,0.069763,0.043350,0.000882,...,3.540941,55.813445,60.112042,311291.107737,23.437463,0.020281,0.012128,13281.774707,269.364861,161.081842
505342,30.0,273881.0,0.025653,0.703435,0.071429,0.038584,0.062193,0.023797,0.036665,0.003309,...,1.663165,21.769511,46.221632,223809.983215,31.729602,0.052569,0.015834,7053.665102,370.800727,111.688520
161769,57.0,279869.0,0.010060,0.639461,0.142857,0.092816,0.033862,0.004956,0.035843,0.006017,...,1.077990,29.777071,70.675701,248973.926419,37.630293,0.055011,0.010822,6616.316468,363.969384,71.602424
373121,53.0,98036.0,0.007876,0.714983,0.115358,0.009532,0.076671,0.000599,0.042532,0.008987,...,2.246182,27.294409,37.592445,406839.226247,38.779985,0.008934,0.004505,10490.958703,93.726164,47.258719
78274,15.0,62667.0,0.013866,0.804472,0.032620,0.014893,0.070666,0.008538,0.048692,0.002898,...,3.415480,63.148214,28.551041,652860.830890,36.914600,0.000469,0.005251,17685.707715,8.302082,92.873085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392347,36.0,174969.0,0.038502,0.730925,0.060810,0.006304,0.039866,0.062002,0.033421,0.002379,...,4.056229,99.548498,61.363280,636550.005003,27.234377,0.007901,0.014552,23373.033857,184.667232,340.116247
172905,45.0,48464.0,0.018002,0.637606,0.036997,0.096724,0.053905,0.079487,0.046055,0.009720,...,1.372961,15.411093,54.701513,185766.235173,31.871833,0.006322,0.007282,5828.539508,36.847321,42.441441
463189,59.0,155265.0,0.021884,0.701784,0.105225,0.010061,0.046157,0.067328,0.010874,0.001325,...,0.859356,7.313871,69.891391,136314.237951,34.700766,0.054364,0.019392,3928.277512,213.558441,76.177444
354735,12.0,48633.0,0.028366,0.739271,0.075997,0.015441,0.019657,0.094274,0.004630,0.009460,...,0.824860,16.225576,35.527261,187099.637813,36.796059,0.005065,0.010922,5084.773837,25.753497,55.534720


In [9]:
mini_sulphur.to_csv('prod_sour_comp_molar_flow_data.csv')

## Модель для определения молекулярного веса сырья и продуктов

In [28]:
mini_data = data.sample(frac=0.85)
len(mini_data)

435880

In [29]:
molar_weight = mini_data[[
    'feed_gas temperature, C', 'feed_gas mass flow, kg/h', 'feed_gas CO2 mol frac', 'feed_gas Methane mol frac',
    'feed_gas Ethane mol frac', 'feed_gas Propane mol frac', 'feed_gas i-Butane mol frac', 'feed_gas n-Butane mol frac',
    'feed_gas i-Pentane mol frac', 'feed_gas n-Pentane mol frac', 'feed_gas H2S mol frac', 'feed_gas H2O mol frac',
    'feed_gas MDEAmine mol frac', 'feed_gas molar flow, kgmol/h', 'feed_gas molecular weight',
    'lean_amine temperature, C', 'lean_amine mass flow, kg/h', 'lean_amine CO2 mol frac',
    'lean_amine Methane mol frac', 'lean_amine Ethane mol frac', 'lean_amine Propane mol frac',
    'lean_amine i-Butane mol frac', 'lean_amine n-Butane mol frac', 'lean_amine i-Pentane mol frac',
    'lean_amine n-Pentane mol frac', 'lean_amine H2S mol frac', 'lean_amine H2O mol frac', 'lean_amine MDEAmine mol frac',
    'lean_amine molar flow, kgmol/h', 'lean_amine molecular weight',
    'rich_amine temperature, C', 'rich_amine mass flow, kg/h', 'rich_amine molecular weight',
    'rich_amine molar flow, kgmol/h', 'sweet_gas molecular weight'
]]

X = molar_weight[[
    'feed_gas temperature, C', 'feed_gas mass flow, kg/h', 'feed_gas CO2 mol frac', 'feed_gas Methane mol frac',
    'feed_gas Ethane mol frac', 'feed_gas Propane mol frac', 'feed_gas i-Butane mol frac', 'feed_gas n-Butane mol frac',
    'feed_gas i-Pentane mol frac', 'feed_gas n-Pentane mol frac', 'feed_gas H2S mol frac', 'feed_gas H2O mol frac',
    'feed_gas MDEAmine mol frac',
    'lean_amine temperature, C', 'lean_amine mass flow, kg/h', 'lean_amine CO2 mol frac',
    'lean_amine Methane mol frac', 'lean_amine Ethane mol frac', 'lean_amine Propane mol frac',
    'lean_amine i-Butane mol frac', 'lean_amine n-Butane mol frac', 'lean_amine i-Pentane mol frac',
    'lean_amine n-Pentane mol frac', 'lean_amine H2S mol frac', 'lean_amine H2O mol frac', 'lean_amine MDEAmine mol frac',
]]

y = molar_weight[[
    'feed_gas molecular weight', 'lean_amine molecular weight', 'rich_amine molecular weight', 'sweet_gas molecular weight'
]]

# Собираем трансформер столбцов
ct = make_column_transformer(
    (MinMaxScaler(), [
    'feed_gas temperature, C', 'feed_gas mass flow, kg/h', 'feed_gas CO2 mol frac', 'feed_gas Methane mol frac',
    'feed_gas Ethane mol frac', 'feed_gas Propane mol frac', 'feed_gas i-Butane mol frac', 'feed_gas n-Butane mol frac',
    'feed_gas i-Pentane mol frac', 'feed_gas n-Pentane mol frac', 'feed_gas H2S mol frac', 'feed_gas H2O mol frac',
    'feed_gas MDEAmine mol frac',
    'lean_amine temperature, C', 'lean_amine mass flow, kg/h', 'lean_amine CO2 mol frac',
    'lean_amine Methane mol frac', 'lean_amine Ethane mol frac', 'lean_amine Propane mol frac',
    'lean_amine i-Butane mol frac', 'lean_amine n-Butane mol frac', 'lean_amine i-Pentane mol frac',
    'lean_amine n-Pentane mol frac', 'lean_amine H2S mol frac', 'lean_amine H2O mol frac', 'lean_amine MDEAmine mol frac',
    ])
)

# Создаем обучающую и тестовые выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Загружаем тренировочную выборку
ct.fit(X_train)

# Нормализуем данные
X_train_normal = ct.transform(X_train)
X_test_normal = ct.transform(X_test)

X_train_normal.shape, y_train.shape

((348704, 26), (348704, 4))

In [32]:
# Set random set
tf.random.set_seed(42)

# 1. Create a model
molar_weight_model = tf.keras.Sequential([
    tf.keras.layers.Dense(26, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(4)
])

# 2. Compile the model
molar_weight_model.compile(
    loss='mae',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['mae']
)

# 3. Fit the model
molar_weight_model.fit(X_train_normal, y_train, epochs=15)

Epoch 1/15
10897/10897 [==============================] - 32s 3ms/step - loss: 0.3366 - mae: 0.3366
Epoch 2/15
10897/10897 [==============================] - 27s 2ms/step - loss: 0.1352 - mae: 0.1352
Epoch 3/15
10897/10897 [==============================] - 28s 3ms/step - loss: 0.1286 - mae: 0.1286
Epoch 4/15
10897/10897 [==============================] - 27s 2ms/step - loss: 0.1183 - mae: 0.1183
Epoch 5/15
10897/10897 [==============================] - 28s 3ms/step - loss: 0.1138 - mae: 0.1138
Epoch 6/15
10897/10897 [==============================] - 27s 2ms/step - loss: 0.1064 - mae: 0.1064
Epoch 7/15
10897/10897 [==============================] - 27s 2ms/step - loss: 0.1044 - mae: 0.1044
Epoch 8/15
10897/10897 [==============================] - 26s 2ms/step - loss: 0.1009 - mae: 0.1009
Epoch 9/15
10897/10897 [==============================] - 26s 2ms/step - loss: 0.0975 - mae: 0.0975
Epoch 10/15
10897/10897 [==============================] - 28s 3ms/step - loss: 0.0959 - mae: 0.0959

In [33]:
y_pred = molar_weight_model.predict(X_test_normal)

2725/2725 [==============================] - 4s 1ms/step


In [34]:
x = 3243
pred = y_pred[x]
pred

array([29.391653, 28.671213, 28.756186, 28.664097], dtype=float32)

In [35]:
test = y_test.iloc[x]
test

feed_gas molecular weight      29.400799
lean_amine molecular weight    28.844534
rich_amine molecular weight    28.884964
sweet_gas molecular weight     28.812946
Name: 127286, dtype: float64

In [36]:
100 - (25.267178 * 100) / 25.478810

0.8306196403992061

In [37]:
molar_weight_model.save('./models/amine_treatment/amine_treatment_molar_weight_model.h5')

In [27]:
mini_molar_weight = molar_weight.sample(frac=0.01)
mini_molar_weight

,"feed_gas temperature, C","feed_gas mass flow, kg/h",feed_gas CO2 mol frac,feed_gas Methane mol frac,feed_gas Ethane mol frac,feed_gas Propane mol frac,feed_gas i-Butane mol frac,feed_gas n-Butane mol frac,feed_gas i-Pentane mol frac,feed_gas n-Pentane mol frac,...,lean_amine H2S mol frac,lean_amine H2O mol frac,lean_amine MDEAmine mol frac,"lean_amine molar flow, kgmol/h",lean_amine molecular weight,"rich_amine temperature, C","rich_amine mass flow, kg/h",rich_amine molecular weight,"rich_amine molar flow, kgmol/h",sweet_gas molecular weight
481165,52.0,113704.0,0.039110,0.683722,0.019643,0.079485,0.086982,0.071729,0.008543,0.007417,...,0.000160,0.782749,0.213695,9250.718178,39.721024,47.855356,374968.491204,39.754154,9432.183958,26.612419
141502,43.0,180816.0,0.035923,0.746856,0.085600,0.030359,0.023647,0.051279,0.005432,0.000906,...,0.000286,0.989468,0.009261,4397.006879,18.982231,42.219311,85092.456982,19.183266,4435.764942,22.862099
243354,35.0,175064.0,0.030521,0.618966,0.160239,0.063034,0.055969,0.049122,0.014598,0.000795,...,0.000274,0.799524,0.199250,3894.655340,38.198245,41.732243,153869.364968,38.244736,4023.282201,26.052115
53792,21.0,101678.0,0.015024,0.651512,0.096782,0.093012,0.003636,0.072973,0.045786,0.005590,...,0.000101,0.828066,0.166886,1691.782553,35.025778,26.678738,63382.825685,35.201221,1800.586021,26.464701
50414,32.0,12274.0,0.021561,0.682571,0.017362,0.074599,0.034665,0.078293,0.048050,0.007875,...,0.000224,0.771418,0.226739,14508.943523,40.995266,45.587122,597024.964079,40.956800,14576.943666,26.560067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395711,36.0,284317.0,0.023935,0.645368,0.060125,0.089620,0.069123,0.077051,0.008231,0.003455,...,0.000331,0.888610,0.108568,1848.945126,29.066844,35.653332,60338.356137,29.895602,2018.302074,27.083897
126488,33.0,185029.0,0.014748,0.689119,0.174222,0.022776,0.024373,0.032820,0.024052,0.002669,...,0.000313,0.796440,0.202424,17706.480339,38.516576,51.749815,691455.078148,38.487514,17965.698744,23.248342
128246,58.0,14824.0,0.016123,0.707491,0.083474,0.047423,0.063816,0.016341,0.048420,0.000464,...,0.000340,0.968342,0.029572,19855.494913,21.057294,55.876320,419177.439708,21.072139,19892.495625,25.128166
37516,45.0,123752.0,0.032740,0.563979,0.151425,0.077542,0.037326,0.064939,0.047364,0.000587,...,0.000130,0.987960,0.007270,25723.813587,18.873329,44.247255,492892.249432,19.012284,25924.936582,28.271857


In [13]:
mini_molar_weight.to_csv('molar_weight_data.csv')

## Модель для определения состава насыщенного амина

In [11]:
mini_data = data.sample(frac=0.75)
len(mini_data)

384600

In [12]:
list(mini_data.columns)

['feed_gas temperature, C',
 'feed_gas pressure, kPa',
 'feed_gas mass flow, kg/h',
 'feed_gas molecular weight',
 'feed_gas Mass density, kg/m3',
 'feed_gas actual liquid flow, m3/h',
 'feed_gas actual vapour flow, m3/h',
 'feed_gas CO2 mol frac',
 'feed_gas Methane mol frac',
 'feed_gas Ethane mol frac',
 'feed_gas Propane mol frac',
 'feed_gas i-Butane mol frac',
 'feed_gas n-Butane mol frac',
 'feed_gas i-Pentane mol frac',
 'feed_gas n-Pentane mol frac',
 'feed_gas H2S mol frac',
 'feed_gas H2O mol frac',
 'feed_gas MDEAmine mol frac',
 'feed_gas H2S ppm',
 'feed_gas CO2 ppm',
 'Absorber number of stages',
 'lean_amine temperature, C',
 'lean_amine pressure, kPa',
 'lean_amine mass flow, kg/h',
 'lean_amine molecular weight',
 'lean_amine Mass density, kg/m3',
 'lean_amine actual liquid flow, m3/h',
 'lean_amine actual vapour flow, m3/h',
 'lean_amine CO2 mol frac',
 'lean_amine Methane mol frac',
 'lean_amine Ethane mol frac',
 'lean_amine Propane mol frac',
 'lean_amine i-Butane

In [73]:
molar_flow = mini_data[[
    'feed_gas temperature, C', 'feed_gas mass flow, kg/h', 'feed_gas molecular weight', 'feed_gas CO2 mol frac',
    'feed_gas Methane mol frac', 'feed_gas Ethane mol frac', 'feed_gas Propane mol frac', 'feed_gas i-Butane mol frac',
    'feed_gas n-Butane mol frac', 'feed_gas i-Pentane mol frac', 'feed_gas n-Pentane mol frac', 'feed_gas H2S mol frac',
    'feed_gas H2O mol frac', 'feed_gas MDEAmine mol frac', 'feed_gas molar flow, kgmol/h',
    'feed_gas H2S molar flow, kgmol/h', 'feed_gas CO2 molar flow, kgmol/h', 'feed_gas H2O molar flow, kgmol/h',
    'feed_gas MDEAmine molar flow, kgmol/h', 'lean_amine temperature, C', 'lean_amine mass flow, kg/h',
    'lean_amine molecular weight', 'lean_amine CO2 mol frac', 'lean_amine Methane mol frac', 'lean_amine Ethane mol frac',
    'lean_amine Propane mol frac', 'lean_amine i-Butane mol frac', 'lean_amine n-Butane mol frac',
    'lean_amine i-Pentane mol frac', 'lean_amine n-Pentane mol frac', 'lean_amine H2S mol frac', 'lean_amine H2O mol frac',
    'lean_amine MDEAmine mol frac', 'lean_amine molar flow, kgmol/h', 'lean_amine H2S molar flow, kgmol/h',
    'lean_amine CO2 molar flow, kgmol/h', 'lean_amine H2O molar flow, kgmol/h', 'lean_amine MDEAmine molar flow, kgmol/h',
    'rich_amine temperature, C', 'rich_amine mass flow, kg/h', 'rich_amine molecular weight',
    'rich_amine molar flow, kgmol/h',  'rich_amine H2S molar flow, kgmol/h', 'rich_amine CO2 molar flow, kgmol/h',
    'rich_amine H2O molar flow, kgmol/h', 'rich_amine MDEAmine molar flow, kgmol/h'
    ]]

X = molar_flow[[
    'feed_gas temperature, C', 'feed_gas mass flow, kg/h', 'feed_gas molecular weight', 'feed_gas CO2 mol frac',
    'feed_gas Methane mol frac', 'feed_gas Ethane mol frac', 'feed_gas Propane mol frac', 'feed_gas i-Butane mol frac',
    'feed_gas n-Butane mol frac', 'feed_gas i-Pentane mol frac', 'feed_gas n-Pentane mol frac', 'feed_gas H2S mol frac',
    'feed_gas H2O mol frac', 'feed_gas MDEAmine mol frac', 'feed_gas molar flow, kgmol/h',
    'feed_gas H2S molar flow, kgmol/h', 'feed_gas CO2 molar flow, kgmol/h', 'feed_gas H2O molar flow, kgmol/h',
    'feed_gas MDEAmine molar flow, kgmol/h', 'lean_amine temperature, C', 'lean_amine mass flow, kg/h',
    'lean_amine molecular weight', 'lean_amine CO2 mol frac', 'lean_amine Methane mol frac', 'lean_amine Ethane mol frac',
    'lean_amine Propane mol frac', 'lean_amine i-Butane mol frac', 'lean_amine n-Butane mol frac',
    'lean_amine i-Pentane mol frac', 'lean_amine n-Pentane mol frac', 'lean_amine H2S mol frac', 'lean_amine H2O mol frac',
    'lean_amine MDEAmine mol frac', 'lean_amine molar flow, kgmol/h', 'lean_amine H2S molar flow, kgmol/h',
    'lean_amine CO2 molar flow, kgmol/h', 'lean_amine H2O molar flow, kgmol/h', 'lean_amine MDEAmine molar flow, kgmol/h',
    'rich_amine temperature, C', 'rich_amine mass flow, kg/h', 'rich_amine molecular weight',
    'rich_amine molar flow, kgmol/h',  'rich_amine H2S molar flow, kgmol/h', 'rich_amine CO2 molar flow, kgmol/h'
]]

y = molar_flow[[
    'rich_amine H2O molar flow, kgmol/h', 'rich_amine MDEAmine molar flow, kgmol/h'
]]

# Собираем трансформер столбцов
ct = make_column_transformer(
    (MinMaxScaler(), [
    'feed_gas temperature, C', 'feed_gas mass flow, kg/h', 'feed_gas molecular weight', 'feed_gas CO2 mol frac',
    'feed_gas Methane mol frac', 'feed_gas Ethane mol frac', 'feed_gas Propane mol frac', 'feed_gas i-Butane mol frac',
    'feed_gas n-Butane mol frac', 'feed_gas i-Pentane mol frac', 'feed_gas n-Pentane mol frac', 'feed_gas H2S mol frac',
    'feed_gas H2O mol frac', 'feed_gas MDEAmine mol frac', 'feed_gas molar flow, kgmol/h',
    'feed_gas H2S molar flow, kgmol/h', 'feed_gas CO2 molar flow, kgmol/h', 'feed_gas H2O molar flow, kgmol/h',
    'feed_gas MDEAmine molar flow, kgmol/h', 'lean_amine temperature, C', 'lean_amine mass flow, kg/h',
    'lean_amine molecular weight', 'lean_amine CO2 mol frac', 'lean_amine Methane mol frac', 'lean_amine Ethane mol frac',
    'lean_amine Propane mol frac', 'lean_amine i-Butane mol frac', 'lean_amine n-Butane mol frac',
    'lean_amine i-Pentane mol frac', 'lean_amine n-Pentane mol frac', 'lean_amine H2S mol frac', 'lean_amine H2O mol frac',
    'lean_amine MDEAmine mol frac', 'lean_amine molar flow, kgmol/h', 'lean_amine H2S molar flow, kgmol/h',
    'lean_amine CO2 molar flow, kgmol/h', 'lean_amine H2O molar flow, kgmol/h', 'lean_amine MDEAmine molar flow, kgmol/h',
    'rich_amine temperature, C', 'rich_amine mass flow, kg/h', 'rich_amine molecular weight',
    'rich_amine molar flow, kgmol/h',  'rich_amine H2S molar flow, kgmol/h', 'rich_amine CO2 molar flow, kgmol/h'
    ])
)

# Создаем обучающую и тестовые выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Загружаем тренировочную выборку
ct.fit(X_train)

# Нормализуем данные
X_train_normal = ct.transform(X_train)
X_test_normal = ct.transform(X_test)

X_train_normal.shape, y_train.shape

((307680, 44), (307680, 2))

In [75]:
# Set random set
tf.random.set_seed(42)

# 1. Create a model
rich_amine_MDEA_H2O_model = tf.keras.Sequential([
    tf.keras.layers.Dense(44, activation='relu'),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(2)
])

# 2. Compile the model
rich_amine_MDEA_H2O_model.compile(
    loss='mae',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['mae']
)

# 3. Fit the model
rich_amine_MDEA_H2O_model.fit(X_train_normal, y_train, epochs=25)

Epoch 1/25
9615/9615 [==============================] - 21s 2ms/step - loss: 339.5964 - mae: 339.5964
Epoch 2/25
9615/9615 [==============================] - 20s 2ms/step - loss: 17.1910 - mae: 17.1910
Epoch 3/25
9615/9615 [==============================] - 19s 2ms/step - loss: 14.0423 - mae: 14.0423
Epoch 4/25
9615/9615 [==============================] - 19s 2ms/step - loss: 12.5483 - mae: 12.5483
Epoch 5/25
9615/9615 [==============================] - 19s 2ms/step - loss: 11.7104 - mae: 11.7104
Epoch 6/25
9615/9615 [==============================] - 18s 2ms/step - loss: 10.8639 - mae: 10.8639
Epoch 7/25
9615/9615 [==============================] - 19s 2ms/step - loss: 10.3327 - mae: 10.3327
Epoch 8/25
9615/9615 [==============================] - 20s 2ms/step - loss: 9.7699 - mae: 9.7699
Epoch 9/25
9615/9615 [==============================] - 20s 2ms/step - loss: 9.3193 - mae: 9.3193
Epoch 10/25
9615/9615 [==============================] - 18s 2ms/step - loss: 9.0369 - mae: 9.0369
Epo

In [76]:
y_pred = rich_amine_MDEA_H2O_model.predict(X_test_normal)

2404/2404 [==============================] - 4s 1ms/step


In [86]:
x = 1105
pred = y_pred[x]
pred

array([12520.134,  2519.256], dtype=float32)

In [87]:
test = y_test.iloc[x]
test

rich_amine H2O molar flow, kgmol/h         12519.618171
rich_amine MDEAmine molar flow, kgmol/h     2519.782083
Name: 275756, dtype: float64

In [88]:
(pred * 100) / test

rich_amine H2O molar flow, kgmol/h         100.004118
rich_amine MDEAmine molar flow, kgmol/h     99.979126
Name: 275756, dtype: float64

In [89]:
rich_amine_MDEA_H2O_model.save('./models/amine_treatment/amine_treatment_rich_amine_MDEA_H2O_molar_flow_model.h5')